In [1]:
from bs4 import BeautifulSoup as soup 
from urllib.request import urlopen as uReq
from urllib.request import Request
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import numpy as np
import os
import re

In [2]:
myurl =  'https://cedh-decklist-database.com'
cards = [] # initializes list of all cards

In [68]:

client = uReq(myurl) # opening connection 
page_html = client.read() # content to variable
client.close() # closes connection 
page_soup = soup(page_html, 'html.parser') # html parsing 
containers = page_soup.findAll("div",{"class": "ddb-section"})
htmls = []
for c in containers:
    if c.get_text().strip() == 'COMPETITIVE':
        x = c.parent.parent.find("ul", {"class": "ddb-decklists"})
        lis = x.findAll('li')
        for li in lis:
            htmls.append(li.a["href"])
len(htmls)

211

In [70]:
#cleaning moxfield primer lists
def second_group(m):
    return m.group(1)
htmls = list(map(lambda x: re.sub(r'(.*)/$|(.*)/primer$',second_group,x ), htmls))
htmls[128:131]

['https://www.moxfield.com/decks/yPLtlCQp00qmtCg3Jis5sw',
 'https://www.moxfield.com/decks/77tKzEJx8UO_YsL1wXKA1A',
 'https://www.moxfield.com/decks/OjY9mvS_eUqlNUqdSpfPTQ']

In [16]:
chromedriver_path=os.path.join(os.getcwd(), "chromedriver.exe")
options = Options()
options.headless = True
for decklist in htmls[:4]:
    if 'moxfield' in decklist: # checks if its a moxfield deck
        driver = webdriver.Chrome(chromedriver_path, options= options)
        driver.get(decklist)
        time.sleep(5) #if you want to wait 3 seconds for the page to load
        page_source = driver.page_source
        driver.close()
        page_soup = soup(page_source, 'html.parser')
        containers = page_soup.findAll("tr",{"class": "table-deck-row"})
        title = page_soup.find("span", {"class": "deckheader-name"}).get_text()
        deck_info = {'Link': decklist, 'Title':title }
        deck_cards = []
        for c in containers:
            c_name = {'Card Name': c.a.get_text()}
            deck_cards.append(c_name)
        deck_info['Cards'] = deck_cards
        cards.append(deck_info)

In [17]:
print(cards[0])

{'Link': 'https://www.moxfield.com/decks/1vqfLIEtWkOnZo-ywKQtKA', 'Title': 'Tymna/Sidar Raza', 'Cards': [{'Card Name': 'Sidar Kondo of Jamuraa'}, {'Card Name': 'Tymna the Weaver'}, {'Card Name': 'Arbor Elf'}, {'Card Name': "Avacyn's Pilgrim"}, {'Card Name': 'Birds of Paradise'}, {'Card Name': 'Cabal Therapist'}, {'Card Name': 'Collector Ouphe'}, {'Card Name': 'Corpse Knight'}, {'Card Name': 'Dark Confidant'}, {'Card Name': 'Dauthi Voidwalker'}, {'Card Name': 'Deathrite Shaman'}, {'Card Name': 'Drannith Magistrate'}, {'Card Name': 'Elesh Norn, Grand Cenobite'}, {'Card Name': 'Elves of Deep Shadow'}, {'Card Name': 'Elvish Mystic'}, {'Card Name': 'Esper Sentinel'}, {'Card Name': 'Fyndhorn Elves'}, {'Card Name': 'Grim Flayer'}, {'Card Name': 'Hope of Ghirapur'}, {'Card Name': 'Leonin Relic-Warder'}, {'Card Name': 'Llanowar Elves'}, {'Card Name': 'Loyal Retainers'}, {'Card Name': 'Opposition Agent'}, {'Card Name': 'Priest of Titania'}, {'Card Name': 'Razaketh, the Foulblooded'}, {'Card Name

In [18]:
df = pd.json_normalize(cards,
    record_path='Cards', meta=['Link', 'Title'])

In [20]:
df['Title'].value_counts()

Tymna/Sidar Raza                 111
[Primer] Rog Turbo Ad Nauseam    111
Frozen Scepter Parun 2.0         105
Niv-Mizzet, Parun                100
Name: Title, dtype: int64

In [84]:
df_1 = df.groupby('Card Name')['Title'].apply(list).reset_index()
df_2 = df.groupby('Card Name')['Link'].apply(list).reset_index()
df_12 = pd.merge(df_1, df_2, on='Card Name')
df_12.head()

,Card Name,Title,Link
0,Academy Ruins,"[Brago, playing cedh with Azorius]",[https://www.moxfield.com/decks/hbLMs7xcfUyun2...
1,Adarkar Wastes,"[Brago, playing cedh with Azorius]",[https://www.moxfield.com/decks/hbLMs7xcfUyun2...
2,Ancient Tomb,"[Brago, playing cedh with Azorius, Evolution: ...",[https://www.moxfield.com/decks/hbLMs7xcfUyun2...
3,Arcane Signet,"[Brago, playing cedh with Azorius, Evolution: ...",[https://www.moxfield.com/decks/hbLMs7xcfUyun2...
4,Archon of Emeria,"[Brago, playing cedh with Azorius]",[https://www.moxfield.com/decks/hbLMs7xcfUyun2...


In [77]:
card_vc = df['Card Name'].value_counts()
card_vc = card_vc.reset_index()
card_vc.rename( columns={'index' :'Card Name', 'Card Name':'Ocurrences'}, inplace=True )#columns names
card_vc

,Card Name,Ocurrences
0,Chrome Mox,5
1,Lotus Petal,5
2,Mana Crypt,5
3,Mox Diamond,5
4,Jeweled Lotus,3
...,...,...
151,Sensei's Divining Top,1
152,Prismatic Vista,1
153,Timetwister,1
154,Waterlogged Grove,1


In [95]:
card_vc.to_json('results/competitiveCards.json', orient='records')
card_vc.to_csv('results/competitive_cards.csv')

In [85]:
full_cards = pd.merge(df_12,card_vc, on = 'Card Name')
full_cards.head()

,Card Name,Title,Link,Ocurrences
0,Academy Ruins,"[Brago, playing cedh with Azorius]",[https://www.moxfield.com/decks/hbLMs7xcfUyun2...,1
1,Adarkar Wastes,"[Brago, playing cedh with Azorius]",[https://www.moxfield.com/decks/hbLMs7xcfUyun2...,1
2,Ancient Tomb,"[Brago, playing cedh with Azorius, Evolution: ...",[https://www.moxfield.com/decks/hbLMs7xcfUyun2...,2
3,Arcane Signet,"[Brago, playing cedh with Azorius, Evolution: ...",[https://www.moxfield.com/decks/hbLMs7xcfUyun2...,2
4,Archon of Emeria,"[Brago, playing cedh with Azorius]",[https://www.moxfield.com/decks/hbLMs7xcfUyun2...,1


In [86]:
full_cards.to_json('results/competitiveCards_full.json', orient='records')

In [96]:
my_collection = pd.read_csv('ArchidektCollection-Sat Jul 10 2021.csv')
my_collection.head()

,Quantity,Name
0,1,"Tasigur, the Golden Fang"
1,1,Witherbloom Apprentice
2,1,Emiel the Blessed
3,1,"Valentin, Dean of the Vein // Lisette, Dean of..."
4,1,Goblin Matron


In [97]:
sd = pd.merge(card_vc,my_collection, how='left', left_on='Card Name', right_on='Name', indicator=True)
sd[sd['_merge']=='left_only'].head(40)

,Card Name,Occurences,Quantity,Name,_merge
34,Imperial Seal,193,NaN,NaN,left_only
52,Timetwister,144,NaN,NaN,left_only
61,Wheel of Fortune,137,NaN,NaN,left_only
82,Volcanic Island,110,NaN,NaN,left_only
90,Badlands,106,NaN,NaN,left_only
93,Necropotence,102,NaN,NaN,left_only
96,Gaea's Cradle,99,NaN,NaN,left_only
121,Cavern of Souls,79,NaN,NaN,left_only
130,Savannah,75,NaN,NaN,left_only
135,Taiga,72,NaN,NaN,left_only


In [66]:
htmls = ['https://www.moxfield.com/decks/1t0Cpm1xqUeyA-6xi2TrGQ','https://www.moxfield.com/decks/_4HeABv9DEeykmnZy1JICg', 'https://www.moxfield.com/decks/Re-_xtEbCkyNdH69vyGR8w','https://www.moxfield.com/decks/TnwluHJbcU2-Ioz8Op4txw']
# htmls = ['https://www.moxfield.com/decks/1t0Cpm1xqUeyA-6xi2TrGQ']
cards = []
chromedriver_path=os.path.join(os.getcwd(), "chromedriver.exe")
options = Options()
options.headless = True
driver = webdriver.Chrome(chromedriver_path, options= options)
for decklist in htmls:
    if 'moxfield' in decklist: # checks if its a moxfield deck
        driver.get(decklist)
        time.sleep(3) #if you want to wait 3 seconds for the page to load
        mode = driver.find_element_by_id('viewMode').get_attribute('value')
        page_source = driver.page_source
        page_soup = soup(page_source, 'html.parser')
        title = page_soup.find("span", {"class": "deckheader-name"}).get_text()
        deck_info = {'Link': decklist, 'Title':title }
        if mode == 'table': 
            containers = page_soup.findAll("tr",{"class": "table-deck-row"})
            deck_cards = []
            #TODO: Find only decklist cards 
            for c in containers:
                c_name = {'Card Name': c.a.get_text()}
                deck_cards.append(c_name)
            deck_info['Cards'] = deck_cards
            cards.append(deck_info)
        elif mode == 'visual':
            containers = page_soup.findAll("div",{"class": "decklist-card-phantomsearch"})
            deck_cards = []
            #TODO: Find only decklist cards 
            for c in containers:
                c_name = {'Card Name': c.get_text()}
                deck_cards.append(c_name)
            deck_info['Cards'] = deck_cards
            cards.append(deck_info)
        elif mode == 'stacks':
            containers = page_soup.findAll("div",{"class": "img-card-stack"})
            deck_cards = []
            #TODO: Find only decklist cards 
            for c in containers:
                c_name = {'Card Name': c.img['alt']}
                deck_cards.append(c_name)
            deck_info['Cards'] = deck_cards
            cards.append(deck_info)

df = pd.json_normalize(cards,
    record_path='Cards', meta=['Link', 'Title'])
df['Title'].value_counts()

Blood Farm            116
BAD Korvold (cEDH)    107
Consultation Zur      100
Marwyn, nerfed         73
Name: Title, dtype: int64